In [ ]:
import os
import json
from dotenv import load_dotenv
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

## Steps to configure the environment file:

Before starting, you need to configure your environment variables. Follow these steps to set up your `.env` file correctly.

- **Step 1:** Create a `.env` file in the current directory.
- **Step 2:** Open the `.env` file and copy the code below into it.
- **Step 3:** Replace the empty single quotes (`''`) with your respective secrets.

---

### Contents of the .env file:

```plaintext
API_KEY='your_api_key_here'
IBM_CLOUD_URL='your_ibm_cloud_url_here'
PROJECT_ID='your_project_id_here'

In [ ]:
#config Watsonx.ai environment
load_dotenv()
project_id = os.environ["PROJECT_ID"]
ibm_cloud_url = os.environ["IBM_CLOUD_URL"]
api_key = os.environ["API_KEY"]

creds = {
        "url": ibm_cloud_url,
        "apikey": api_key 
    }

In [ ]:
def send_to_watsonxai(prompts,
                    model_name="google/flan-ul2",
                    decoding_method="greedy",
                    max_new_tokens=200,
                    min_new_tokens=30,
                    temperature=1.0,
                    repetition_penalty=1.0
                    ):
    '''
   helper function for sending prompts and params to Watsonx.ai
    
    Args:  
        prompts:list list of text prompts
        decoding:str Watsonx.ai parameter "sample" or "greedy"
        max_new_tok:int Watsonx.ai parameter for max new tokens/response returned
        temperature:float Watsonx.ai parameter for temperature (range 0>2)
        repetition_penalty:float Watsonx.ai parameter for repetition penalty (range 1.0 to 2.0)

    Returns: None
        prints response
    '''

    assert not any(map(lambda prompt: len(prompt) < 1, prompts)), "make sure none of the prompts in the inputs prompts are empty"

    # Instantiate parameters for text generation
    model_params = {
        GenParams.DECODING_METHOD: decoding_method,
        GenParams.MIN_NEW_TOKENS: min_new_tokens,
        GenParams.MAX_NEW_TOKENS: max_new_tokens,
        GenParams.RANDOM_SEED: 42,
        GenParams.TEMPERATURE: temperature,
        GenParams.REPETITION_PENALTY: repetition_penalty,
    }


    # Instantiate a model proxy object to send your requests
    model = Model(
        model_id=model_name,
        params=model_params,
        credentials=creds,
        project_id=project_id)

    output = []
    for prompt in prompts:
        o = model.generate_text(prompt)
        output.append(o)
    return output

In [ ]:
# List of models
llama2_chat = "meta-llama/llama-2-70b-chat"
granite="ibm/granite-13b-chat-v1"

# Classification Zero shot learning

## Sentiment analysis

In [ ]:
#Q1 Code - enter prompt and parameters in this cell

system_prompt = """
You are the sentiment analysis program which return json format {"result":"positive"} or {"result": "negative"}, 
You must answer only json format withou any explanation, 
Output Schema: schema = {
      "type": "object",
      "properties": {
        "result": "string"
            }
        }
        """
text = '''
ผมมีสัญญาเงินกู้บ้านกับ Wells Fargo มาหลายปีแล้ว จ่ายทุกเดือนเร็วกว่ากำหนัดอย่างน้อย 7-10 วัน ระหว่าง XX/XX/XXXX และ XX/XX/XXXX ผมจ่าย {$3000.00} ทุกเดือน 
ในปี XXXX ผมได้รับโทรศัพท์จาก Wells Fargo ว่าการชำระเงินรายเดือนของฉันไม่ถูกต้อง หลังจากการสนทนายาวนาน ผมยอมจ่ายเพิ่มเติม {$750.00} 
เพื่อทำให้บัญชีปัจจุบันและชำระ $ XXXX ต่อมา ในวันที่ XX/XX/XXXX ผมได้รับจดหมายจาก XXXX 
ว่าสัญญาเงินกู้ของผมอยู่ในสถานะค้างชำระและแนะนำให้ผมดำเนินการทันที หลังจากการสนทนายาวนาน 
ผมจึงทราบว่าในวันที่ XX/XX/XXXX ธนาคารได้รับการชำระเงินของผมแล้ว แต่เนื่องจากจำนวนเงินน้อยกว่าที่พวกเขาต้องการ 
พวกเขาไม่ได้จ่ายเงินในสัญญาเงินกู้ของผม แต่พวกเขาได้นำจำนวนเงินทั้งหมดไปชำระเงินต้น พวกเขาไม่เคยบอกผมเรื่องนี้ และพวกเขาได้รายงานข้อมูลของผมให้กับหน่วยงานเครดิต 
'''

llama2_prompt = f"""
    <s>[INST] <<SYS>>
        {{
        {system_prompt}
        }}
    <</SYS>>
        {{
        {text}
        }}
        [/INST]
"""
#Complete your prompt here 

response_g = send_to_watsonxai(prompts=[llama2_prompt],
                            model_name=granite
                            )

response_l = send_to_watsonxai(prompts=[llama2_prompt],
                            model_name=llama2_chat
                            )



In [ ]:
print(response_g)
print(response_l)

## Classification

บทความต่อไปนี้เป็นการร้องเรียนของผู้บริโภค 
การร้องเรียนเกี่ยวกับหนึ่งในตัวเลือกเหล่านี้: บัตรเครดิต, รายงานเครดิต, สินเชื่อที่จดทะเบียนและสินเชื่อ, ธนาคารขายปลีก, หรือการค้าสินทรัพย์ 
โปรดอ่านย่อด้านล่างและกำหนดว่าการร้องเรียนนี้เกี่ยวข้องกับตัวเลือกใด.

In [ ]:
system_prompt = '''
You are the sentence topic classifier can understand Thai.
Given only 5 classes that you have to classify the input sentence: [credit cards, credit reporting, mortgages and loans, retail banking, debt collection].
Example output: ['mortgages and loans']
Example output: ['credit cards']
Example output: ['credit reporting']
Example output: ['retail banking']
Example output: ['animal']
Classify the input sentence into only 1 class less than 10 words
'''


text = '''
ผมกู้บ้านกับ Wells Fargone มาหลายปีแล้ว จ่ายทุกเดือนเร็วกว่ากำหนัดอย่างน้อย 7-10 วัน ระหว่าง XX/XX/XXXX และ XX/XX/XXXX ผมจ่าย {$3000.00} ทุกเดือน 
ในปี XXXX ผมได้รับโทรศัพท์จาก Wells Fargo ว่าการชำระเงินรายเดือนของฉันไม่ถูกต้อง หลังจากการสนทนายาวนาน ผมยอมจ่ายเพิ่มเติม {$750.00} 
เพื่อทำให้บัญชีปัจจุบันและชำระ $ XXXX ต่อมา ในวันที่ XX/XX/XXXX ผมได้รับจดหมายจาก XXXX 
ว่าสัญญาเงินกู้ของผมอยู่ในสถานะค้างชำระและแนะนำให้ผมดำเนินการทันที หลังจากการสนทนายาวนาน 
ผมจึงทราบว่าในวันที่ XX/XX/XXXX ธนาคารได้รับการชำระเงินของผมแล้ว แต่เนื่องจากจำนวนเงินน้อยกว่าที่พวกเขาต้องการ 
พวกเขาไม่ได้จ่ายเงินในสัญญาเงินกู้ของผม แต่พวกเขาได้นำจำนวนเงินทั้งหมดไปชำระเงินต้น พวกเขาไม่เคยบอกผมเรื่องนี้ และพวกเขาได้รายงานข้อมูลของผมให้กับหน่วยงานเครดิต 
'''

llama2_prompt = f"""
    <s>[INST] <<SYS>>
        {{
        {system_prompt}
        }}
    <</SYS>>
        {{
        {text}
        }}
        [/INST]
"""


response_l = send_to_watsonxai(prompts=[llama2_prompt],
                            model_name=llama2_chat
                            )

In [ ]:
print(response_l)

# Oneshot learning

In [ ]:
system_prompt = """
You are the sentence entity extractor that can understand Thai.
You have to follow the example
        """

example_entity = """
sentence: ฉันถอนเงิน 100 บาท จากธนาคารไทยพาณิชย์ ด้วยเบอร์ 0649611111 ให้สมชาย
entity: {
  "Amount": "100 บาท",
  "Bank Name": "ธนาคารไทยพาณิชย์",
  "Phone Number": "0649611111",
  "Name": "สมชาย"
}

"""

text = '''
วันนี้ฉันจ่ายเงิน 500 บาท ที่ธนาคารกรุงไทย โดยใช้หมายเลขโทรศัพท์ 099221xxyy ให้กับนายชนาธิป
'''

llama2_prompt = f"""
    <s>[INST] <<SYS>>
        {{
        {system_prompt}
        EXAMPLE:
        {example_entity}
        }}
        
    <</SYS>>
        {{
        {text}
        }}
        [/INST]
"""

response_l = send_to_watsonxai(prompts=[llama2_prompt],
                            model_name=llama2_chat
                            )

In [ ]:
print(response_l)

## Two shot learning

In [ ]:
system_prompt = """
You are the sentence entity extractor that can understand Thai.
You have to follow the output JSON format
"""

example_entity = """
sentence: ฉันถอนเงิน 100 บาท จากธนาคารไทยพาณิชย์ ด้วยเบอร์ 0649611111 ให้สมชาย
entity: {
  "Amount": "100 บาท",
  "Bank Name": "ธนาคารไทยพาณิชย์",
  "Phone Number": "0649611111",
  "Name": "สมชาย"
}
"""

example_entity2 = """
sentence: ขอบคุณที่ติดต่อกับฉันค่ะ บัตรเครดิตของฉันถูกยกเลิกเมื่อฉันอยู่ในต่างประเทศเป็นวันหยุดค่ะ หมายเลขบัตรเครดิตของฉันคือ 39482374859 และชื่อบัญชีของฉันคือ ฟิล สมิธ และที่อยู่ของฉันคือ 34 ซิตีเวล, เมลเบิร์น, 3000 วิซซาดี ค่ะ วิธีการติดต่อที่ฉันชอบคืออีเมล: phil.smith@email.com ค่ะ
entity: {
  "Credit Card Number": "39482374859",
  "Account Name": "ฟิล สมิธ",
  "Address": "34 ซิตีเวล, เมลเบิร์น, 3000 วิซซาดี",
  "Email": "phil.smith@email.com"
}
"""

text2 = """ สวัสดี ฉันชื่อ นางสมหญิง แซ่ตั้ง ฉันเขียนถึงคุณเพื่อรายงานรายการทำรายการที่ไม่ได้รับอนุญาตบนบัตรเครดิตของฉันค่ะ
เมื่อวันที่ 30 มีนาคม 2023 ฉันสังเกตรายการชาร์จ 1,000 ดอลลาร์ในรายการบัตรเครดิตของฉันที่ไม่ได้รับอนุญาตในวันนั้น ธุรกรรมนี้ทำการที่ร้านอาหารในนิวยอร์ก 
ในขณะที่ฉันอยู่ในแคลิฟอร์เนียในวันนั้นค่ะ ฉันกังวลเกี่ยวกับความปลอดภัยของบัญชีของฉันและฉันคิดว่านี่ควรจะได้รับการตรวจสอบโดยด่วนค่ะ
โปรดติดต่อฉันที่หมายเลขโทรศัพท์ 123-456-7890 หรืออีเมลที่ somyhing.st@email.com เพื่อให้ข้อมูลปรับปรุงเกี่ยวกับการสืบสวนค่ะ
ธนาคาร Tisco หมายเลขบัตรเครดิตของฉันคือ 3572267594198019 และหมายเลขประจำตัวประชาชนของฉันคือ 175-43-9027 ฉันยังคาดหวังที่จะได้รับข้อมูลจากคุณด่วนค่ะ
"""

llama2_prompt = f"""
    <s>[INST] <<SYS>>
        {{
        {system_prompt}
        EXAMPLE1:
        {example_entity}
        EXAMPLE2:
        {example_entity2}

        }}
        
    <</SYS>>
        {{
        sentence:
        {text2}
        entity:
        }}
        [/INST]
"""


response_l = send_to_watsonxai(prompts=[llama2_prompt],
                            model_name=llama2_chat,
                              max_new_tokens=250
                            )

In [ ]:
print(response_l)